# Transformer 구조의 이해
<center>

![](./img/transformer.png)
<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    "fig 1: Transformer Model"
  </figcaption>
  
</center>

위 fig 1 은, Attention is all you need 논문에서 제안된 transformer 모델의 architecture 입니다.\
encoder & decoder 구조로, 많은 분야에 큰 영향을 주었죠.

특히, transformer 의 꽃이라고 말 할 수 있는 부분은 Attention 구조입니다.\
해당 구조를 통해서 많은 성능 개선과 아이디어들이 등장할 수 있게 되었죠. 

다양한 논문들이 Transformer 를 변형하여 새로운 task 를 다루게 되었습니다.\
대표적으로 다음과 같은 변형 방법이 있습니다. 

- Transformer 의 encoder 만을 사용하는 방법 
- Transformer 의 decoder 만을 사용하는 방법

Transformer 의 encoder 만을 사용하는 방법으로 대표적인 예시는 BERT (bidirectional Encoder Representations from Transformers) 가 있겠네요.\
또한, **ViT 도 encoder 만을 사용하는 방법**입니다.

Transformer 의 decoder 만을 사용하는 방법으로 대표적인 예시는 GPT 입니다. 


><details><summary>
>
>### BERT & GPT
></summary>
> BERT 와 GPT 는 모두 Transformer 의 일부 형태를 사용합니다.
>
> 다만, 모델의 구조만을 보면 서로 동일하다고 생각하기 쉽습니다.
>
> 이 두 모델을 정확하게 이해하려면, 각 모델이 **학습하는 과정**과 **inference 과정**을 비교해 보는 것이 좋습니다. 
><center>
> 
> ![](./img/BertAndGpt.png)
> <figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
>     from : Book [Foundation Models for Natural Language Processing Pre-trained Language Models Integrating Media]
>   </figcaption>
>   
> </center>
>
> #### BERT
> BERT 는 전체 시퀀스를 한번에 입력 받습니다. 
> 다만, 특정 문장을 [MASK] 처리하여 해당 [MASK] 에 들어갈 문장을 학습하고, 추론합니다. 
>
> #### GPT
> GPT 는 앞선 문장들을 입력으로 받으며, 다음에 등장할 단어를 추론하는 방식으로 학습합니다. 
> 그러므로, [the mouse eats cheese] 라는 문장을 아래와 같이 여러번의 입력으로 나누고 recursive 하게 다음 단어를 추론하게 됩니다.
> - the
> - the mouse
> - the mouse eats 
> - ...
> 
></details>


이전 장에서 ViT 의 입력으로 사용되는 이미지를 patch 단위로 나누었고, vector 로 embedding 했습니다. 

이제, embedding 한 vector 를 Transformer Encoder 의 입력으로 넣기 위해 Transformer Encoder 의 전체 구조를 알아봅시다.

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary


from utils import Image_Embedding # 이전 장의 image embedding

In [ ]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, 
                 emb_size: int = 768,
                 drop_out: float = 0.):
        super(TransformerEncoderBlock, self).__init__()

        self.normalizeLayer = Normalize()
        self.multiHeadAttention = MultiHeadAttention()
        self.residualAdd = ResidualAdd()
        self.